In [59]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [60]:
# 라이브러리 가져오기
import requests
import pandas as pd
import os
import dotenv

In [61]:
dotenv.load_dotenv()
my_key = os.getenv('KAKAO_RESTFULL_KEY')

In [62]:
# Kakao Map API 통해 위도, 경도 데이터 가져오는 함수를 정의
def get_geocoding(place):
    try:
        url = f"https://dapi.kakao.com/v2/local/search/address"
        headers = {"Authorization": f"KakaoAK {my_key}"}
        params = {
            'query' : place
        }
        response = requests.get(url, headers=headers,params=params)
        result = response.json()
        if result["documents"]:
            return result["documents"][0]["y"], result["documents"][0]["x"]
        else:
            return None,None
    except:
        return None,None

In [73]:
url = f"https://dapi.kakao.com/v2/local/search/address"
headers = {"Authorization": f"KakaoAK {my_key}"}
params = {
    'query' : '서울특별시 종로구 상하동'
}
response = requests.get(url, headers=headers,params=params)
result = response.json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [63]:
lat = []  #위도
lng = []  #경도

# 장소(주소) 리스트
places = ["서울특별시 종로구 세종대로 175", 
          "서울특별시 서초구 서초동 700", 
          "부산광역시 해운대구 해운대해변로 264"]

i=0
for place in places:   
    i = i + 1
    try:
        print(i, place)
        # get_geocoding 함수의 리턴값 호출하여 geo_location 변수에 저장
        place_lat, place_lon = get_geocoding(place)
        lat.append(place_lat)
        lng.append(place_lon)
        
    except:
        lat.append('')
        lng.append('')

# 데이터프레임으로 변환하기
df = pd.DataFrame({'위도':lat, '경도':lng}, index=places)

df

1 서울특별시 종로구 세종대로 175
2 서울특별시 서초구 서초동 700
3 부산광역시 해운대구 해운대해변로 264


,위도,경도
서울특별시 종로구 세종대로 175,37.5718478584908,126.976168275947
서울특별시 서초구 서초동 700,37.4810862955299,127.015245160054
부산광역시 해운대구 해운대해변로 264,35.1591069824231,129.160283786856


In [64]:
df = pd.read_csv('https://raw.githubusercontent.com/pia222sk20/python/main/data/sample.csv'
                 ,encoding='cp949')
df_simple = df.loc[:,['상호명','도로명주소']]
df_simple.head()

,상호명,도로명주소
0,하나산부인과,경기도 안산시 단원구 달미로 10
1,타워광명내과의원,서울특별시 강남구 언주로30길 39
2,조정현신경외과의원,경기도 시흥시 중심상가로 178
3,한귀원정신과의원,부산광역시 수영구 수영로 688
4,더블유스토어수지점,경기도 용인시 수지구 문정로 32


In [65]:
#위도 경고를 추가  apply
# df_simple['위경도'] = df_simple['도로명주소'].apply(lambda x : get_geocoding(x))
df_simple['도로명주소'].index

RangeIndex(start=0, stop=91335, step=1)

In [66]:
import random
radom_index = random.sample(range(91335),50)
radom_index[:10]

[19252, 52166, 66250, 73379, 22156, 78011, 64324, 79414, 15360, 86921]

In [ ]:
df_simple = df_simple.dropna()  # 데이터 중에서 None 또는 NaN 없는 데이터 row를 삭제

In [68]:
df_simple_50 =  df_simple.iloc[radom_index]
results = [ get_geocoding(addr) for addr in df_simple_50['도로명주소'].values]
results[:10]

[('35.8425738690814', '127.140054344904'),
 ('37.8687445127498', '127.714867915848'),
 ('37.4050705280976', '126.909517658269'),
 ('34.9553665923882', '127.692842356847'),
 ('36.8335382459996', '127.137145348787'),
 ('37.5686335795096', '127.184751315964'),
 ('37.6509740519626', '127.142176639613'),
 ('35.1765258048493', '129.085439791267'),
 ('37.5101663829521', '127.11797580084'),
 ('34.9569538715888', '127.483864759802')]

In [69]:
df_simple_50.loc[:,'위도'], df_simple_50.loc[:,'경도'] = zip(*results)

C:\Users\playdata2\AppData\Local\Temp\ipykernel_13380\497364495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simple_50.loc[:,'위도'], df_simple_50.loc[:,'경도'] = zip(*results)
C:\Users\playdata2\AppData\Local\Temp\ipykernel_13380\497364495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simple_50.loc[:,'위도'], df_simple_50.loc[:,'경도'] = zip(*results)


In [70]:
df_simple_50.head()

,상호명,도로명주소,위도,경도
19252,고려마취통증의학과의원,전라북도 전주시 덕진구 백제대로 646,35.8425738690814,127.140054344904
52166,백세약국,강원도 춘천시 스포츠타운길 530,37.8687445127498,127.714867915848
66250,정한의원,경기도 안양시 만안구 양화로136번길 5,37.4050705280976,126.909517658269
73379,w-store광양서울약국,전라남도 광양시 진등6길 51,34.9553665923882,127.692842356847
22156,수의원,충청남도 천안시 서북구 원두정3길 37,36.8335382459996,127.137145348787


In [ ]:
# 지도
%pip install folium

In [82]:
for i in df_simple_50:
    print(i)    

상호명
도로명주소
위도
경도


In [87]:
for idx, row in df_simple_50.iterrows():
    print(idx, row['위도'], row['경도'])
    break

19252 35.8425738690814 127.140054344904


In [89]:
import folium

m = folium.Map(location=(35.8425738690814, 127.140054344904))  # 최초지도 로드할때. 중심좌표
for idx, row in df_simple_50.iterrows():
    folium.Marker(
        location=[ row['위도'], row['경도'] ],
        popup=row['상호명'],
        tooltip=row['상호명'][-2:]
    ).add_to(m)    

m.save('map.html')